# Loading Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import keras
from keras.layers import Conv2D, MaxPool2D,Dropout,Flatten,Dense
from keras.preprocessing import image 
from keras.models import Sequential
import tensorflow as tf
from tensorflow.keras import models
import datetime


In [ ]:
# Use the GPU on a Mac M1
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    tf.config.set_visible_devices(physical_devices[0], 'GPU')
print(physical_devices)

# Loading Data

In [ ]:
train_data_gen = image.ImageDataGenerator(rescale= 1./255)
train = train_data_gen.flow_from_directory(directory="archive/casting_data/casting_data/train/" ,
 target_size=(256,256) , batch_size=32, class_mode = 'binary')

In [ ]:
train_data_gen = image.ImageDataGenerator(rescale= 1./255)
test = train_data_gen.flow_from_directory(directory="archive/casting_data/casting_data/test" ,
 target_size=(256,256) , batch_size=32, class_mode = 'binary')

# Verifying labels

In [ ]:
train.class_indices


In [ ]:
test.class_indices

# Dataset examples

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline


# Set the class labels
class_labels = {0: 'def_front', 1: 'ok_front'}

# Get a batch of images and labels
images, labels = next(train)

# Select 5 images with 'ok_front' label and 5 images with 'def_front' label
ok_front_images = images[labels == 1][:5]
ok_front_labels = labels[labels == 1][:5]
def_front_images = images[labels == 0][:5]
def_front_labels = labels[labels == 0][:5]

# Concatenate the images and labels
images = np.concatenate([ok_front_images, def_front_images])
labels = np.concatenate([ok_front_labels, def_front_labels])

# Plot the images and their labels
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(15, 6))

for i, ax in enumerate(axes.flat):
    ax.imshow(images[i])
    ax.set_title(class_labels[labels[i]])
    ax.axis('off')
    
plt.tight_layout()
plt.show()


# Transfer Learning - loading feature extraction layers

In [ ]:
from tensorflow.keras.applications import Xception
xcept = Xception(input_shape = (256, 256, 3), include_top = False, weights = 'imagenet')

# Add Head Layers to network

In [ ]:
#Feature layers not be be trained again
%load_ext tensorboard

# Clear any logs from previous runs
#rm -rf ./logs/


for layer in xcept.layers:
    layer.trainable = False


model=keras.Sequential([
    xcept,
    keras.layers.Flatten(),
    keras.layers.Dense(units=256, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(units=1, activation="sigmoid"),
])


model.compile(optimizer="adam",loss='binary_crossentropy',metrics=['accuracy'])

#Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Model Summary

In [ ]:
model.summary()

# Training

In [ ]:
history = model.fit_generator(train,epochs=15,steps_per_epoch=20,validation_data=test,validation_steps=len(test), callbacks=[tensorboard_callback])

# Visualization

In [ ]:
print(model.history.history.keys())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))

# Plot loss
sns.lineplot(x=range(1, len(model.history.history['loss'])+1),
             y=model.history.history['loss'], color='blue', ax=ax1)
sns.lineplot(x=range(1, len(model.history.history['val_loss'])+1),
             y=model.history.history['val_loss'], color='orange', ax=ax1)
ax1.set_title('Training and Validation Loss', fontweight='bold', fontsize=14)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend(['train', 'validation'])

# Plot accuracy
sns.lineplot(x=range(1, len(model.history.history['accuracy'])+1),
             y=model.history.history['accuracy'], color='blue', ax=ax2)
sns.lineplot(x=range(1, len(model.history.history['val_accuracy'])+1),
             y=model.history.history['val_accuracy'], color='orange', ax=ax2)
ax2.set_title('Training and Validation Accuracy', fontweight='bold', fontsize=14)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.legend(['train', 'validation'])

plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define colors for the lines
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

# Create subplots for loss and accuracy
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))

# Plot loss
sns.lineplot(x=range(1, len(model.history.history['loss'])+1),
             y=model.history.history['loss'], color=colors[0], ax=ax1)
sns.lineplot(x=range(1, len(model.history.history['val_loss'])+1),
             y=model.history.history['val_loss'], color=colors[1], ax=ax1)
ax1.set_title('Training and Validation Loss', fontweight='bold', fontsize=14)
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax1.legend(labels=['train loss', 'val loss'], loc='upper right',
           bbox_to_anchor=(1, 1), frameon=False,
           labelcolor=[colors[0], colors[1]])

# Plot accuracy
sns.lineplot(x=range(1, len(model.history.history['accuracy'])+1),
             y=model.history.history['accuracy'], color=colors[2], ax=ax2)
sns.lineplot(x=range(1, len(model.history.history['val_accuracy'])+1),
             y=model.history.history['val_accuracy'], color=colors[3], ax=ax2)
ax2.set_title('Training and Validation Accuracy', fontweight='bold', fontsize=14)
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax2.legend(labels=['train accuracy', 'val accuracy'], loc='upper right',
           bbox_to_anchor=(1, 1), frameon=False,
           labelcolor=[colors[2], colors[3]])

plt.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Get predicted classes
y_pred = model.predict(test)
y_pred = (y_pred > 0.5).astype(int)

# Create confusion matrix
cm = confusion_matrix(test.classes, y_pred)

# Plot confusion matrix
sns.set(font_scale=1.4)
sns.heatmap(cm, annot=True, annot_kws={"size": 16}, cmap='Blues', fmt='g')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

In [ ]:
from sklearn.metrics import f1_score
import numpy as np

# Get predictions for test set
y_pred = model.predict(test)

# Convert probabilities to binary predictions
y_pred = np.round(y_pred)

# Get true labels for test set
y_true = test.labels

# Calculate f1 score
f1 = f1_score(y_true, y_pred)

print("F1 Score:", f1)

# Plot f1 score
plt.bar(['F1 Score'], [f1])
plt.title("F1 Score")
plt.show()

# Export model

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
#model.save_weights("model.h5")